# **(ARL) ASSOCIATION RULE LEARNING PROJECT USING APRIORI ALGORITHM**





*The Apriori algorithm iteratively creates product combinations according to a specified "support" value and finally extracts the association rules by filtering the product combinations according to this support value. In order to understand how Apriori algorithm works, please check the excel file in the folder that I calculated manually*

DATASET:   https://archive.ics.uci.edu/ml/datasets/Online+Retail+II 

*  Online Retail II dataset includes the sales of a UK-based online retail store between the Year 2009-2011.

***The main goal of the project is to apply Association Rule Learning analysis(ARL) using the Apriori algorithm for "France" for the years betweeen 2010-2011 to the online retail_II data set. ***



*   First, I needed to deduplicate the retail data set. I've created the invoice product (basked product) matrix over the transaction data as shown below.

In [ ]:
# Invoice Product(Basket product) example
# Every row should represent a unique invoice, and this will be treated as basket product as shown below

# Description   NINE DRAWER OFFICE TIDY   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET
# Invoice
# 536370                              0                                 1                       0
# 536852                              1                                 0                       1
# 536974                              0                                 0                       0
# 537065                              1                                 0                       0
# 537463                              0                                 0                       1

# this structure above has been the core point- structure - fundamentals of complex recommendation systems.

In [4]:
# 1. Data Preprocessing
#   1. Missing Values, Outliers etc.
#   2. Creating the Invoice product (basket product) matrix
# 2. Establishing Association Rules

In [33]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

df_ = pd.read_excel("/content/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


# Data Preprocessing

In [35]:
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)


 # CRM specialized Function (Please see in CRM projects)
def crm_data_prep(dataframe):
    dataframe.dropna(axis=0, inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    return dataframe


def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [36]:
check_df(df)

##################### Shape #####################
(541910, 8)
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00   2.55      17850.0  United Kingdom
1  536365     71053                 WHITE METAL LANTERN         6 2010-12-01 08:26:00   3.39      17850.0  United Kingdom
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00   2.75      17850.0  United Kingdom
##################### Tail #####################
       Invoice StockCode                      Description  Quantity         InvoiceDate  Price  Cu

In [37]:
df = crm_data_prep(df)
check_df(df)

##################### Shape #####################
(397925, 9)
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity         InvoiceDate  Price  Customer ID         Country  TotalPrice
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER       6.0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom       15.30
1  536365     71053                 WHITE METAL LANTERN       6.0 2010-12-01 08:26:00   3.39      17850.0  United Kingdom       20.34
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER       8.0 2010-12-01 08:26:00   2.75      17850.0  United Kingdom       22.00
##################### Tail #####################
       Invoice Stock

In [38]:
# For France
df_fr = df[df["Country"] == "France"]
check_df(df_fr)

##################### Shape #####################
(8343, 9)
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
   Invoice StockCode                 Description  Quantity         InvoiceDate  Price  Customer ID Country  TotalPrice
26  536370     22728   ALARM CLOCK BAKELIKE PINK      24.0 2010-12-01 08:45:00   3.75      12583.0  France        90.0
27  536370     22727   ALARM CLOCK BAKELIKE RED       24.0 2010-12-01 08:45:00   3.75      12583.0  France        90.0
28  536370     22726  ALARM CLOCK BAKELIKE GREEN      12.0 2010-12-01 08:45:00   3.75      12583.0  France        45.0
##################### Tail #####################
       Invoice StockCode                      Description  Quantity         Invoic

In [39]:
# The number of unique product
df_fr["StockCode"].nunique()  # 1522

1522

In [40]:
# The number of unique customer
df_fr["Customer ID"].nunique()  # 87

87

In [41]:
# How many unique invoice- transaction?
df_fr["Invoice"].nunique()  # 389


389

In [42]:
# Number of transactions(invoice) for each customer, number of products purchased and average amount paid
df_fr.groupby("Customer ID").agg({"Invoice": "nunique",
                                  "StockCode": "count",
                                  "TotalPrice": "mean"}).sort_values("Invoice", ascending=False).head()

,Invoice,StockCode,TotalPrice
Customer ID,,,
12682.0,31,525,23.406133
12681.0,22,638,21.610564
12437.0,18,200,24.757050
12683.0,15,344,24.641192
12583.0,15,247,29.431964


In [43]:
# Singularize the invoices and sum the quantities
df_fr.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"}).head(100)

Quantity
Invoice StockCode          
536370  10002          48.0
        21035          18.0
        21724          12.0
        21731          24.0
        21791          24.0
...                     ...
537065  47566           5.0
        84678           6.0
        85227          12.0
        84279P          4.0
        84870B          4.0

[100 rows x 1 columns]

In [44]:
df_fr.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"}).unstack().iloc[0:5, 0:5]

Quantity                        
StockCode    10002 10120 10125 10135 11001
Invoice                                   
536370        48.0   NaN   NaN   NaN   NaN
536852         NaN   NaN   NaN   NaN   NaN
536974         NaN   NaN   NaN   NaN   NaN
537065         NaN   NaN   NaN   NaN   NaN
537463         NaN   NaN   NaN   NaN   NaN

In [45]:
# observing - validating the logic of unstack
df_fr[df_fr["StockCode"] == 10002].sort_values(["Invoice", "Quantity"], ascending=[True, False])
df_fr.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"}).unstack().iloc[0:15, 0:15]
df[(df["StockCode"] == 10002) & (df["Invoice"] == 536370)]  # validation

#           Quantity
# StockCode    10002 10120 10125 10135 11001
# Invoice
# 536370        48.0   NaN   NaN   NaN   NaN
# 536852         NaN   NaN   NaN   NaN   NaN
# 536974         NaN   NaN   NaN   NaN   NaN
# 537065         NaN   NaN   NaN   NaN   NaN
# 537463         NaN   NaN   NaN   NaN   NaN

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
31,536370,10002,INFLATABLE POLITICAL GLOBE,48.0,2010-12-01 08:45:00,0.85,12583.0,France,40.8


In [46]:
# Nan values will be filled with 0 (without inplace= True, permanent changes will be done with function)
df_fr.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"}).unstack().fillna(0).iloc[0:5, 0:5]

Quantity                        
StockCode    10002 10120 10125 10135 11001
Invoice                                   
536370        48.0   0.0   0.0   0.0   0.0
536852         0.0   0.0   0.0   0.0   0.0
536974         0.0   0.0   0.0   0.0   0.0
537065         0.0   0.0   0.0   0.0   0.0
537463         0.0   0.0   0.0   0.0   0.0

In [47]:
# non-null values will be filled with 1 and non values => 0
df_fr.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"}).unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0).iloc[0:5, 0:5]
# While "apply" method in python works for either column or row whereas applymap method works for all

Quantity                        
StockCode    10002 10120 10125 10135 11001
Invoice                                   
536370           1     0     0     0     0
536852           0     0     0     0     0
536974           0     0     0     0     0
537065           0     0     0     0     0
537463           0     0     0     0     0

*As shown above, I transformed the data into technically expected matrix form, product basket(Invoice product form).
I placed each invoice to the index, each stock code to the column and whenever there exists a product on the intersection 
point of invoice and quantity and consequently I filled with 1 and the rest with 0*

So now let's write a function to automate creating invoice product dataframe practices for association rule learning models.

In [48]:
def create_invoice_product_df(dataframe):
    return dataframe.groupby(["Invoice", "Description"])["Quantity"].sum().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [49]:
fr_inv_pro_df = create_invoice_product_df(df_fr)
fr_inv_pro_df.head()
fr_inv_pro_df.iloc[0:5, 0:5]


Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON
Invoice,,,,,
536370,0,0,0,0,1
536852,0,0,0,0,0
536974,0,0,0,0,0
537065,0,0,0,0,0
537463,0,0,0,0,0


In [50]:
# How many unique items are in each invoice?
print(fr_inv_pro_df.sum(axis=1).head())

Invoice
536370    20
536852     7
536974    16
537065    63
537463    41
dtype: int64


In [51]:
# How many unique baskets are there in each product?
print(fr_inv_pro_df.sum(axis=0).head())

Description
 50'S CHRISTMAS GIFT BAG LARGE      1
 DOLLY GIRL BEAKER                  9
 I LOVE LONDON MINI BACKPACK        5
 NINE DRAWER OFFICE TIDY            1
 SET 2 TEA TOWELS I LOVE LONDON     7
dtype: int64


# Establishing the Association Rules

In [52]:
frequent_itemsets = apriori(fr_inv_pro_df, min_support=0.01, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False)

rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CHARLOTTE BAG DOLLY GIRL DESIGN),( DOLLY GIRL BEAKER),0.066838,0.023136,0.012853,0.192308,8.311966,0.011307,1.209450
1,( DOLLY GIRL BEAKER),(CHARLOTTE BAG DOLLY GIRL DESIGN),0.023136,0.066838,0.012853,0.555556,8.311966,0.011307,2.099614
2,(DOLLY GIRL CHILDRENS BOWL),( DOLLY GIRL BEAKER),0.046272,0.023136,0.017995,0.388889,16.808642,0.016924,1.598504
3,( DOLLY GIRL BEAKER),(DOLLY GIRL CHILDRENS BOWL),0.023136,0.046272,0.017995,0.777778,16.808642,0.016924,4.291774
4,(DOLLY GIRL CHILDRENS CUP),( DOLLY GIRL BEAKER),0.041131,0.023136,0.015424,0.375000,16.208333,0.014473,1.562982


*Antecedents represents the product that observed first.  Antecedent support means the probability of observing this product.*

*Support represents the probability of observing antecedents and consequents together.*

*Confidence : the probability that (let say) product Y will be sold when product X is sold*

*Lift : with the purchase of product X, the probability of purchase of product Y increases by "lift" times.*

*Conviction: Expected frequency of product X without product Y*

In [53]:
rules.sort_values("lift", ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1025639,"(RED RETROSPOT MINI CASES, CHILDRENS CUTLERY D...","(DOLLY GIRL CHILDRENS CUP, LUNCH BAG APPLE DES...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1206591,"(JUMBO BAG APPLES, ALARM CLOCK BAKELIKE GREEN,...","(DOLLY GIRL CHILDRENS CUP, LUNCH BOX WITH CUTL...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1206677,"(RED RETROSPOT MINI CASES, SKULL LUNCH BOX WIT...","(DOLLY GIRL CHILDRENS CUP, ALARM CLOCK BAKELIK...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1206678,"(ALARM CLOCK BAKELIKE RED , SKULL LUNCH BOX WI...","(DOLLY GIRL CHILDRENS CUP, ALARM CLOCK BAKELIK...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1206679,"(ALARM CLOCK BAKELIKE RED , RED RETROSPOT MINI...","(DOLLY GIRL CHILDRENS CUP, ALARM CLOCK BAKELIK...",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf


When association rules are identified by the support values above, and findings are sorted by "lift", the most associated products are shown above. According to these findings, remarkable amount of data-driven decisions can be made regarding marketing,designing of e-commerce categories and recommendation systems.